In [1]:
import pickle
import re

from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np

from utils import extract_data, basic_sep, arrays_to_complex
from qubit_measurement_analysis.data import SingleShot, ShotCollection

from qubit_measurement_analysis.classification.temporal_post_processor import TPP
from qubit_measurement_analysis.classification import GaussianMixtureModels
from qubit_measurement_analysis.classification.metrics import Metrics as M 

import scienceplots

plt.style.use(["science", "no-latex"])
# plt.style.use("default")


fontsize = 12
plt.rcParams.update({"font.size": fontsize, "lines.linewidth": 1.5})

In [2]:
qubit = 1-1

data_path1 = Path(
    r"C:\Users\nokol\OneDrive\Deep Learning\state_discriminator\pkl_data\4_qubits\4Q_readout\singleshots_sep_Q1-Q2-Q3-Q4"
)
data_path1 = Path(r"/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4")

# Define the regular expression pattern
pattern1 = re.compile(r"(0000|1111)_Q1\.pkl$")
pattern16 = re.compile(r"_Q1\.pkl$")

# retrieve paths of selected files
INPUT_FILES16 = sorted([file for file in data_path1.glob("*.pkl") if pattern16.search(file.name)])

INPUT_FILES16

[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0000_Q1.pkl'),
 PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0001_Q1.pkl'),
 PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0010_Q1.pkl'),
 PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0011_Q1.pkl'),
 PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0100_Q1.pkl'),
 PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0101_Q1.pkl'),
 PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0110_Q1.pkl'),
 PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots

In [3]:
from typing import Iterable
def collection_from_files(input_files, trc_st=0, trc_en=1000, samples_slice=slice(5000), qubit_idx=None): 
    collection = ShotCollection()
    for file in input_files:
        with open(file, "rb") as f:
            data = pickle.load(f)
        keys, values = extract_data(file)
        # print(f"qubits: {keys}\tprep state: {values}")
        if qubit_idx:
            keys = keys[qubit_idx:qubit_idx+1]
            values = values[qubit_idx:qubit_idx+1]
            
        qubits_classes = {key: value for key, value in zip(keys, values)}
        meas_time = data[3]
        if ~isinstance(data[2], Iterable):
            freqs = {keys[0]: data[2]}
        else: 
            freqs = {key: value for key, value in zip(keys, data[2])}
        in_phase_state = data[0]
        quadrature_state = data[1]

        in_phase_state, quadrature_state, meas_time = basic_sep(
        in_phase_state, quadrature_state, meas_time, trc_st, trc_en
        )
        values = arrays_to_complex(
            in_phase_state[samples_slice], quadrature_state[samples_slice]
        )
        
        # print(f"values Mb: {values.nbytes * 1e-6}, meas_time Mb: {meas_time.nbytes * 1e-6}")

        collection.extend(
            [SingleShot(value=value, qubits_classes=qubits_classes) for value in values]
        )
    return collection, freqs, meas_time

        

In [4]:
TRC_ST = 200
TRC_EN = 1192

LAMBDA = 700

## State 0

In [5]:
current_files = INPUT_FILES16[:8]

current_states = [str(string)[-11:-7] for string in current_files]
current_states

['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111']

In [6]:
import fastcluster as fc

for i in range(0, len(current_files), 2):
    curr = current_files[i : i + 2]
    curr_states = [str(string)[-11:-7] for string in curr]
    print(curr)
    collection, freqs, meas_time = collection_from_files(
        curr, trc_st=TRC_ST, trc_en=TRC_EN, samples_slice=slice(10000)
    )

    collection = (
        collection.mean_centring_all()
        .demodulate_all(freqs, meas_time, "clockwise")
        .compute()
    )
    print(collection.shape)

    z_naming = f"Z_{'_'.join(curr_states)}_lamb_{LAMBDA}.npy"
    print(z_naming)

    collection_smooth = collection.whittaker_eilers_smoother_all(LAMBDA, 2).compute()
    collection_smooth.to("cuda")
    distance_matrix = collection_smooth.sspd_to(collection_smooth, method="self_cross_product")

    Z = fc.linkage(distance_matrix[0], method="complete")

    collection_smooth.to("cpu")
    collection.to("cpu")

    np.save(z_naming, np.array(Z))
    

[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0000_Q1.pkl'), PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0001_Q1.pkl')]
(20000, 1, 992)
Z_0000_0001_lamb_700.npy
[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0010_Q1.pkl'), PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0011_Q1.pkl')]
(20000, 1, 992)
Z_0010_0011_lamb_700.npy
[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0100_Q1.pkl'), PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_0101_Q1.pkl')]
(20000, 1, 992)
Z_0100_0101_lamb_700.npy
[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q

## State 1

In [7]:
current_files = INPUT_FILES16[8:]

current_states = [str(string)[-11:-7] for string in current_files]
current_files, current_states

([PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1000_Q1.pkl'),
  PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1001_Q1.pkl'),
  PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1010_Q1.pkl'),
  PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1011_Q1.pkl'),
  PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1100_Q1.pkl'),
  PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1101_Q1.pkl'),
  PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1110_Q1.pkl'),
  PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/sin

In [8]:
import fastcluster as fc

for i in range(0, len(current_files), 2):
    curr = current_files[i : i + 2]
    curr_states = [str(string)[-11:-7] for string in curr]
    print(curr)
    collection, freqs, meas_time = collection_from_files(
        curr, trc_st=TRC_ST, trc_en=TRC_EN, samples_slice=slice(10000)
    )

    collection = (
        collection.mean_centring_all()
        .demodulate_all(freqs, meas_time, "clockwise")
        .compute()
    )
    print(collection.shape)

    z_naming = f"Z_{'_'.join(curr_states)}_lamb_{LAMBDA}.npy"
    print(z_naming)

    collection_smooth = collection.whittaker_eilers_smoother_all(LAMBDA, 2).compute()
    collection_smooth.to("cuda")
    distance_matrix = collection_smooth.sspd_to(collection_smooth, method="self_cross_product")

    Z = fc.linkage(distance_matrix[0], method="complete")

    collection_smooth.to("cpu")
    collection.to("cpu")

    np.save(z_naming, np.array(Z))

[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1000_Q1.pkl'), PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1001_Q1.pkl')]
(20000, 1, 992)
Z_1000_1001_lamb_700.npy
[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1010_Q1.pkl'), PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1011_Q1.pkl')]
(20000, 1, 992)
Z_1010_1011_lamb_700.npy
[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1100_Q1.pkl'), PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q2-Q3-Q4_1101_Q1.pkl')]
(20000, 1, 992)
Z_1100_1101_lamb_700.npy
[PosixPath('/home/kolya/github/data/snowdrop4Q/4Q_readout/singleshots_sep_Q1-Q2-Q3-Q4/singleshots_sep_Q1-Q